<a href="https://colab.research.google.com/github/EngrTeekay/Machine_learning_Cnn/blob/main/project2_catvsdog_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Image classification with Pytorch

In [ ]:
#Import the libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import shutil
import os
from PIL import Image
import gdown

In [ ]:
#mount drive and download dataset

!gdown https://drive.google.com/uc?id=1RCYJeTbxMBKLVEv2inJHGlADvyi3xvof
!unzip dog&cat.zip



Downloading...
From: https://drive.google.com/uc?id=1RCYJeTbxMBKLVEv2inJHGlADvyi3xvof
To: /content/dog&cat.zip
 45% 370M/818M [00:09<00:11, 39.6MB/s]Traceback (most recent call last):
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.7/ssl.py", line 1071, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.7/ssl.py", line 929, in read
    return self._sslobj.read(len, buffer)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/gdown/cli.py", line 166, in main
    resume=args.continue_,
  File "/usr/local/lib/python3.7/dist-packages/gdown/download.py", line 257, in download
    for chunk in res.iter_content(chunk_size=CHUNK_SIZE):
  File "/usr/local/lib/python3.7/dist-packages/requests/models.py", line 751, 

In [ ]:
#creating image directory
#splitting dataset into train and test folder


os.mkdir('train')
os.mkdir('train/cat')
os.mkdir('train/dog')
os.mkdir('test')
os.mkdir('test/cat')
os.mkdir('test/dog')
imgs=os.listdir('/content/images/images')
for i in imgs:
  if i[0].isupper():
    old_path='/content/images/images/' + i
    new_path='train/cat/' + i
    shutil.copy(old_path, new_path)
  else:
    old_path='/content/images/images/' + i
    new_path='train/dog/' + i
    shutil.copy(old_path, new_path)



In [ ]:
#check old directory and move folders to new dir
for i in os.listdir('train/dog')[4000:]:
  old_path='train/dog/' + i
  new_path= 'test/dog/' + i
  shutil.move(old_path, new_path)


for i in os.listdir('train/cat')[2000:]:
  old_path='train/cat/' + i
  new_path= 'test/cat/' + i
  shutil.move(old_path, new_path)
  

In [ ]:
#Get the total length of dataset

#len(os.listdir('train/dog/')) + len(os.listdir('test/dog/')) + len(os.listdir('train/cat/')) + len(os.listdir('test/cat/'))
Get length of images dataset
len(os.listdir('/content/images/images'))

7393

In [ ]:

#data preprocessing and conversion to tensor format

transform = transforms.Compose([
                                  
                                  transforms.RandomResizedCrop(255),
                                  transforms.RandomHorizontalFlip(255), #flip image
                                  transforms.ToTensor(), #convert to tensor format
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) #normalize tensors
])

 train_dataset = datasets.ImageFolder('/content/train', transform)
 test_dataset=   datasets.ImageFolder('/content/test', transform)

 batch_size =16

# #train and test dataloader
 train_set= DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)
 test_set =DataLoader(dataset=test_dataset, shuffle=True, batch_size= batch_size)

#if cuda availabe then use device as cuda else use gpu
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)



Using device: cuda


In [ ]:
# Model Architecture
class Convnet(nn.Module):
    def __init__(self):
        super(Convnet,self).__init__()
        
        #convolutional layer 1
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        #convolutional layer 2
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
            )
        #convolutional layer 3
        self.layer3 = nn.Sequential(
             nn.Conv2d(32,64, kernel_size=3, padding=0, stride=2),
             nn.BatchNorm2d(64),
             nn.ReLU(),
             nn.MaxPool2d(2)
            )
        
        #Linear layer 1
        self.fc1 = nn.Linear(3*3*64,10)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(10,2)
        self.relu = nn.ReLU()
        
        #forward propagation
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return out

model=Convnet().to(device)

In [ ]:
#Define Loss and optimize functions ##use SGD or Adam
criterion = nn.CrossEntropyLoss() #crossEntropy for classification problem
optimizer =optim.Adam(model.parameters(), lr=0.001) #lr,weight_decay=hyperparameter


In [ ]:
#Training and validation of the model
for epoch in range(1, 20):
  epoch_loss = 0
  epoch_accuracy = 0
  for data, target in train_set:
      optimizer.zero_grad()
      data=data.to(device)
      target=target.to(device)
      output = model(data)
      _, preds = torch.max(output, 1)
      loss = criterion(output, target)

      #backward-pass
      loss.backward()

      # Update the parameters
      optimizer.step()
      
      # Update the Training loss
      acc = ((output.argmax(dim=1) == target).float().mean())
      epoch_accuracy += acc/len(train_set)
      epoch_loss += loss/len(train_set)
  print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch, epoch_accuracy,epoch_loss))

  #Testing and validation of model accuracy, loss
  with torch.no_grad():
        epoch_val_accuracy=0
        epoch_val_loss =0
        for data, label in test_set:
            data = data.to(device)
            label = label.to(device)
            
            val_output = model(data)
            val_loss = criterion(val_output,label)
            
            
            acc = ((val_output.argmax(dim=1) == label).float().mean())
            epoch_val_accuracy += acc/ len(test_set)
            epoch_val_loss += val_loss/ len(test_set)
            
        print('Epoch : {}, val_accuracy : {}, val_loss : {}'.format(epoch, epoch_val_accuracy,epoch_val_loss))

Epoch : 1, train accuracy : 0.6643966436386108, train loss : 0.6109201908111572
Epoch : 1, val_accuracy : 0.7130682468414307, val_loss : 0.5802928805351257
Epoch : 2, train accuracy : 0.6665509939193726, train loss : 0.595743715763092
Epoch : 2, val_accuracy : 0.722301185131073, val_loss : 0.5636072754859924
Epoch : 3, train accuracy : 0.6721404194831848, train loss : 0.5919072031974792
Epoch : 3, val_accuracy : 0.7173295617103577, val_loss : 0.5658845901489258
Epoch : 4, train accuracy : 0.6845125555992126, train loss : 0.5784310698509216
Epoch : 4, val_accuracy : 0.696732759475708, val_loss : 0.5701094269752502
Epoch : 5, train accuracy : 0.6886785626411438, train loss : 0.574515700340271
Epoch : 5, val_accuracy : 0.702414870262146, val_loss : 0.5656699538230896
Epoch : 6, train accuracy : 0.6883069276809692, train loss : 0.572537899017334
Epoch : 6, val_accuracy : 0.7208807468414307, val_loss : 0.5478041172027588
Epoch : 7, train accuracy : 0.7040507197380066, train loss : 0.5653372

In [ ]:
#Save my model
torch.save(model,'model.pth')

In [ ]:
#Testing data here
import PIL
model=torch.load('/content/model.pth')
image_path='/content/3426.jpg'
image=PIL.Image.open(image_path)
image=transform(image).unsqueeze(0)
model.cpu()
model.eval()
pred=torch.argmax(model(image))

#Output prediction
if pred.item()==0:
   print('The image is a CAT')
elif pred.item()== 1:
   print('The image is a DOG')
else:
 print("Unable to identify image")

The image is a DOG
